In [1]:
import pandas as pd

In [ ]:
df=pd.read_csv('goup_project_sample_v3.csv')
df.head()

,date,ret_eom,permno,SHRCD,EXCHCD,mspread,year,month,size_port,RF,...,betadown_252d,bidaskhl_21d,corr_1260d,betabab_1260d,rmax5_rvol_21d,age,qmj,qmj_prof,qmj_growth,qmj_safety
0,20000131,20000131,10001,11,3,0.031007,2000,1,S,0.0041,...,0.598605,0.007935,0.040985,0.100657,1.064359,168,-0.320694,0.362350,-0.211910,-0.430240
1,20000131,20000131,10002,11,3,0.014570,2000,1,S,0.0041,...,-0.737535,0.044064,0.095459,0.299299,0.836199,168,0.662709,0.161284,-0.342450,1.604983
2,20000131,20000131,10009,11,3,0.015596,2000,1,S,0.0041,...,-0.118137,0.019126,0.115088,0.182300,0.710678,168,0.673152,0.047310,-0.070927,1.467627
3,20000131,20000131,10012,11,3,0.035910,2000,1,S,0.0041,...,2.297222,0.032665,0.140769,0.966027,1.612729,180,1.184870,1.313923,1.492074,-0.381070
4,20000131,20000131,10016,11,3,0.014664,2000,1,S,0.0041,...,0.269846,0.012475,0.145160,0.253578,1.552247,432,1.270156,1.295644,1.653944,-0.308918


In [5]:
mkt=pd.read_csv('mkt_ind1.csv')
mkt.head()

,RF,year,month,sp_ret
0,0.0041,2000,1,-0.050904
1,0.0043,2000,2,-0.020108
2,0.0047,2000,3,0.096720
3,0.0046,2000,4,-0.030796
4,0.0050,2000,5,-0.021915
